This script converts dicoms of the functional sessions ('ses-se', 'ses-me') to Nifti in bids-format

In [1]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
import json
from glob import glob
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import re
#%matplotlib inline

In [2]:
# global options/settings
ext = '.nii'
root_raw_data = '/home/stevenm/MultiEchoEPISeq/data/raw/dicom'

### Dicom to nifti conversion

In [110]:
dcm2nii_dict = {
    'sub-01': {
        'ses-se': 'bd0499ff-e709a583-4e1d5117-f7108599-75f20412.zip',
        'ses-me': 'f4ca606c-44fa5000-cd822d6e-f4afaec5-347723d8.zip'
    },
    'sub-02': {
        'ses-me': 'aa992687-6be019be-f5573e8f-79174cd0-17bfe594.zip',
        'ses-se': '6113a245-de260a27-4e914f58-1e86e635-00fd7938.zip'
    },
    'sub-03': {
        'ses-se': 'e8d86fc2-d960e9aa-c21a86ca-1527352d-e4f7779a.zip',
        'ses-me': '7b68e417-d3c63cec-9aeabf5c-4ac1ecbe-b4f2cd53.zip'
    },
    'sub-04': {
        'ses-me': '6994f153-caf09c0a-155a6e7d-bba5024a-1169c647.zip',
        'ses-se': '7dcd5ff5-068d24b0-d8e5938e-baf89605-b50777d3.zip'
    },
    'sub-05': {
        'ses-se': '7a931048-ebba8ca7-2dcb5706-fb5ab19b-8ffcdd2f.zip',
        'ses-me': '5a305d22-e4c90ca7-43b6bf58-4df4d956-a3c7898e.zip'
    },
    'sub-06': {
        'ses-me': 'f3871193-d66be461-753f80f1-17358447-a366f18f.zip',
        'ses-se': '85f77949-7e77a5e6-89d92727-8f8cf99b-6c42b18a.zip'
    },
    'sub-07': {
        'ses-se': '60a39eec-8a8904db-c51af47b-0ce64312-ce9a65f1.zip',
        'ses-me': '34cc68bb-f862cff0-641c6c79-d4b40211-a647420f.zip'
    },
    'sub-08': {
        'ses-me': '00fab4d5-76881a0f-29ae2073-9ae3e64d-97a2b83f.zip',
        'ses-se': '661928f3-b6012b3e-5d26d11b-3a8bdccb-2b6aedb3.zip'
    },
    'sub-09': {
        'ses-se': 'c7f6968f-c1f83abc-c9e3572f-bc85146f-9604f8b1.zip',
        'ses-me': '64f7bc2a-f0327997-d29ed8a0-870dc7d4-dac3b553.zip'
    },
    'sub-10': {
        'ses-me': '894335be-6d4c927f-1b5b3c25-a94489c0-6cda18ec.zip',
        'ses-se': '542d7eab-bc4947a7-ac2ba2c3-0bfb6a27-bcb07700.zip'
    },
    'sub-11': {
        'ses-se': '80f57752-8f671d5b-40fdfde7-cad60037-2322692c.zip',
        'ses-me': '31bd7265-01496729-aedc2355-4e69cc03-3bd5600c.zip'
    },
    'sub-12': {
        'ses-me': '8a970dad-ac1217ad-5fca9e7e-b947cdf3-8e119b7c.zip',
        'ses-se': 'e187c97e-68ecca03-7b19801d-bde41eaf-434fcf5d.zip'
    },
    'sub-13': {
        'ses-se': '5a217187-3550aaa7-fc2e459c-7f9a13f5-bcc99461.zip',
        'ses-me': 'aa6bc799-4c08de45-28267adb-98f5c6f6-5e2b4299.zip'
    },
    'sub-14': {
        'ses-me': '9d1f88fd-b6e119d3-61859c3c-b7b0d49c-4eea3e59.zip',
        'ses-se': '78280164-f8e44d4d-6c1d17fc-75470488-9bf9c092.zip'
    },
    'sub-15': {
        'ses-se': 'de00c317-c8015163-6539667d-4e18a7c0-57591eec.zip',
        'ses-me': '15484.08'  # this original zip file was corrupted and replaced by a new download in a slightly different format
    },
    'sub-16': {
        'ses-me': '53dad91d-567101c3-f64ac66b-f1362639-4278144a.zip',
        'ses-se': '894ca696-00b7f86c-9e74dbed-9982f17b-872a13b5.zip'
    },
    'sub-17': {
        'ses-se': '4edc6117-509509d0-9fb174ed-0fb2120d-e1fde59d.zip',
        'ses-me': '9448464d-3cf9f4f4-7922dd68-ed2047a6-943e54bd.zip'
    },
    'sub-18': {
        'ses-me': '1f88a319-802adc86-f2154ca5-3f4b9d76-62056c19.zip',
        'ses-se': 'bea9deb0-2622a572-c0630182-ab3d9d0d-954536c0.zip'
    }
}

## 1. Dicom to Nifti

In [111]:
#

from_files = []
to_files = []

for sub_idx, session_dict in dcm2nii_dict.items():
    for session_name, raw_dir in session_dict.items():
        if not (sub_idx == 'sub-13' and session_name=='ses-se'):
            continue
        nii_data_location = './data/raw/nifti/%s/%s' %(sub_idx, session_name)
        
        from_files.append(os.path.join(root_raw_data, raw_dir))
        to_files.append(os.path.join(nii_data_location, '%s_%s' %(sub_idx, session_name)))
        
# check for existence of output files
to_process = []
for from_fn, to_fn in zip(from_files, to_files):
    to_process.append([from_fn, to_fn])

cmds = []
for i, fns in enumerate(to_process):
    out_name = os.path.basename(fns[1])
    out_dir = os.path.abspath(os.path.dirname(fns[1]))
    tmp_dir = os.path.dirname(fns[0]) + '/tmp_' + str(i)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    if fns[0].endswith('.zip'):
        #unzip to tmp folder, convert to nifti, remove unzipped tmp folder (for some reason dcm2niix on Linux doesnt accept zip files)
        cmds.append('unzip {} -d {} && dcm2niix -6 -b y -i y -f %s_%d_echo-%e -o {} {} && rm -r {}'.format(fns[0],
                                                                                                        tmp_dir,
                                                                                                        out_dir, 
                                                                                                        tmp_dir,
                                                                                                        tmp_dir))
    else:
        cmds.append("unzip '{}/*.zip' -d {} && dcm2niix -6 -b y -i y -f %s_%d_echo-%e -o {} {} && rm -r {}".format(fns[0],
                                                                                                                tmp_dir,
                                                                                                                out_dir, 
                                                                                                                tmp_dir,
                                                                                                                tmp_dir))
cmds

['unzip /home/stevenm/MultiEchoEPISeq/data/raw/dicom/5a217187-3550aaa7-fc2e459c-7f9a13f5-bcc99461.zip -d /home/stevenm/MultiEchoEPISeq/data/raw/dicom/tmp_0 && dcm2niix -6 -b y -i y -f %s_%d_echo-%e -o /home/stevenm/MultiEchoEPISeq/data/raw/nifti/sub-13/ses-se /home/stevenm/MultiEchoEPISeq/data/raw/dicom/tmp_0 && rm -r /home/stevenm/MultiEchoEPISeq/data/raw/dicom/tmp_0']

In [112]:
def run_cmd(x):
    os.system(x)

from multiprocessing import Pool

with Pool(10) as p:
    p.map(run_cmd, cmds)

# 2. Nifti to BIDS

In [113]:
nii2bids_dict = {
    'sub-01': { 
         'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S1s1-stop_fmri.log',
             'anat': {
                 'T1map': '11_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '12_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '9_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '10_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '5_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '8_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '6_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '6_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '7_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S1s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-02': { 
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S2s1-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S2s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
            'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
            'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        }
    },
    'sub-03': { 
         'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S3s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S3s2-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-04': { 
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S4s1-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
         'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S4s2-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-05': { 
         'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S5s1-stop_fmri.log',
             'func': {
                 'task-stop_run-1_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '8_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S5s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-06': {
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S6s1-stop_fmri.log',
             'func': {
                 'task-stop_run-1_echo-1_bold': '5_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '6_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '10_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '7_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '7_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '8_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
         'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S6s2-stop_fmri.log',
             'anat': {
                 'T1map': '11_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '12_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '9_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '10_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '5_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '8_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '6_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '6_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '7_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-07': {
        'ses-se': {
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S7s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-08': {
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S8s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S8s2-stop_fmri.log',
#              'anat': {
#                  'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
#                  'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
#                  'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
#                  'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        }

    },
    'sub-09': {
        'ses-se': {
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S9s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-10': {
         'ses-me': {
             'behavioral_log_file': './data/raw/behavior/S10s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
             'behavioral_log_file': './data/raw/behavior/S10s2-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
    },
    'sub-11': {
        'ses-se': {
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
#             'behavioral_log_file': './data/raw/behavior/S11s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-12': {
#          'ses-me': {
#               # consider this run a failure, unfortunately
#              'behavioral_log_file': './data/raw/behavior/S10s1-stop_fmri.log',
#              'anat': {
#                  'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
#                  'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
#                  'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
#                  'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
#              'func': {
#                  'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
#                  'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
#                  'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
#              'fmap': {
#                  'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
#                  'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
#                  'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
#              },
        'ses-se': {
#             'behavioral_log_file': './data/raw/behavior/S10s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
    },
    'sub-13': {
        'ses-se': {
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
#             'behavioral_log_file': './data/raw/behavior/S11s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
         },
    },
    'sub-14': {
         'ses-me': {
#              'behavioral_log_file': './data/raw/behavior/S10s1-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
#             'behavioral_log_file': './data/raw/behavior/S10s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
    },
    'sub-15': {
        'ses-se': {
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
#             'behavioral_log_file': './data/raw/behavior/S11s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-16': {
         'ses-me': {
#             'behavioral_log_file': './data/raw/behavior/S10s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
#             'behavioral_log_file': './data/raw/behavior/S10s2-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
    },
    'sub-17': {
        'ses-se': {
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
         'ses-me': {
#             'behavioral_log_file': './data/raw/behavior/S11s2-stop_fmri.log',
             'anat': {
                 'T1map': '9_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '10_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '11_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
    },
    'sub-18': {
         'ses-me': {
#              'behavioral_log_file': './data/raw/behavior/S10s1-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_echo-1_bold': '3_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-2_echo-1_bold': '4_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1',
                 'task-stop_run-3_echo-1_bold': '7_MultiEcho_2D_TR3000_TE9p66-24p87-40p08_1p6_70sli_bFOV_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
             },
        'ses-se': {
#             'behavioral_log_file': './data/raw/behavior/S10s2-stop_fmri.log',
             'anat': {
                 'T1map': '10_mp2rage_whole-brain_0p7_T1_Images_echo-1',
                 'T1w': '11_mp2rage_whole-brain_0p7_UNI_Images_echo-1',
                 'inv-1_MPRAGE': '8_mp2rage_whole-brain_0p7_INV1_echo-1',
                 'inv-2_MPRAGE': '9_mp2rage_whole-brain_0p7_INV2_echo-1'},
             'func': {
                 'task-stop_run-1_bold': '3_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-2_bold': '4_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1',
                 'task-stop_run-3_bold': '7_SingleEcho_2D_TR3000_TE14p0_1p6_70sl_echo-1'}, 
             'fmap': {
                 'magnitude1': '5_fieldmap56_DeltaTE1p02ms_echo-1',
                 'magnitude2': '5_fieldmap56_DeltaTE1p02ms_echo-2',
                 'phasediff': '6_fieldmap56_DeltaTE1p02ms_echo-2_ph'}
        },
    }
}

In [13]:
#!ls ./data/raw/nifti/sub-06/ses-me/
# ./data/raw/nifti/sub-02/ses-me/
# 10_mp2rage_whole-brain_0p7_T1_Images_echo-1.nii.gz

In [121]:
root_dir = './data/raw/nifti'
bids_dir = './data/raw/bids'

from_files = []
to_files = []


for sub_idx, session_dict in nii2bids_dict.items():
    for session_name, session_subdict in session_dict.items():
        if not (sub_idx == 'sub-13' and session_name=='ses-se'):
            continue
        nii_data_location = root_dir + '/%s/%s' %(sub_idx, session_name)
        bids_data_location = bids_dir + '/%s/%s' %(sub_idx, session_name)
        for data_type in ['anat', 'func', 'fmap']:
            if not data_type in session_subdict.keys():
                continue
            
            for to_name, from_name in session_subdict[data_type].items():
                from_files.append(os.path.join(nii_data_location, from_name))
                to_files.append(os.path.join(bids_data_location, data_type, 
                                             '%s_%s_%s' %(sub_idx, session_name, to_name)))
                if session_name == 'ses-me' and 'MultiEcho' in from_name:
                    from_files.append(os.path.join(nii_data_location, from_name.replace('_echo-1', '_echo-2')))
                    from_files.append(os.path.join(nii_data_location, from_name.replace('_echo-1', '_echo-3')))
                    to_files.append(bids_data_location + '/' + data_type + '/%s_%s_%s' %(sub_idx, session_name, to_name.replace('echo-1', 'echo-2')))
                    to_files.append(bids_data_location + '/' + data_type + '/%s_%s_%s' %(sub_idx, session_name, to_name.replace('echo-1', 'echo-3')))

# check if to_files already exist
from_name_final = []
to_name_final = []
for from_name, to_name in zip(from_files, to_files):
#     if 'echo-' in to_name:
#         if os.path.exists(to_name.replace('_echo-1', '').replace('_echo-2', '').replace('_echo-3', '') + '.nii.gz'):
#             continue
#     if os.path.exists(to_name + '.nii.gz'):
#         continue
    from_name_final.append(from_name)
    to_name_final.append(to_name)

# Rename everything
for from_name, to_name in zip(from_name_final, to_name_final):
    # check if dir exists
    if not os.path.exists(os.path.dirname(to_name)):
        os.makedirs(os.path.dirname(to_name))
    
    if not os.path.exists(from_name + ext):
        print('Could not move %s; already moved?' %from_name)
    else:
        os.rename(from_name + ext, to_name + ext)
        os.rename(from_name + '.json', to_name + '.json')

In [120]:
!ls -la ./data/raw/bids/sub-13/ses-se/anat

total 8
drwxr-xr-x 2 marcus users 4096 nov 22 12:25 .
drwxr-xr-x 5 marcus users 4096 nov 22 12:25 ..


In [17]:
to_name_final

['./data/raw/bids/sub-13/ses-se/anat/sub-13_ses-se_T1map',
 './data/raw/bids/sub-13/ses-se/anat/sub-13_ses-se_T1w',
 './data/raw/bids/sub-13/ses-se/anat/sub-13_ses-se_inv-1_MPRAGE',
 './data/raw/bids/sub-13/ses-se/anat/sub-13_ses-se_inv-2_MPRAGE',
 './data/raw/bids/sub-13/ses-se/func/sub-13_ses-se_task-stop_run-1_bold',
 './data/raw/bids/sub-13/ses-se/func/sub-13_ses-se_task-stop_run-2_bold',
 './data/raw/bids/sub-13/ses-se/func/sub-13_ses-se_task-stop_run-3_bold',
 './data/raw/bids/sub-13/ses-se/fmap/sub-13_ses-se_magnitude1',
 './data/raw/bids/sub-13/ses-se/fmap/sub-13_ses-se_magnitude2',
 './data/raw/bids/sub-13/ses-se/fmap/sub-13_ses-se_phasediff']

In [122]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
# list_files('./data/raw/bids')

##### Add metadata to json files
For T1map, T1w (MP2RAGE derivatives), for functional scans, and for the fieldmap (phasediff should contain both echotimes)

In [5]:
with open('./data/raw/bids/sub-01/ses-me/fmap/sub-01_ses-me_phasediff.json', 'r') as f:
    d = json.load(f)

In [6]:
d

{'Modality': 'MR',
 'MagneticFieldStrength': 6.98,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Investigational_Device_7T',
 'InstitutionName': 'MPI_Leipzig',
 'InstitutionalDepartmentName': 'Department',
 'InstitutionAddress': 'Stephanstrasse_1a_Leipzig-FCAF96-_Leipzig_DE_04103',
 'DeviceSerialNumber': '18919',
 'StationName': 'SEPTEMSYS',
 'PatientPosition': 'HFS',
 'ProcedureStepDescription': 'TA_VB17_FATAG1',
 'SoftwareVersions': 'syngo_MR_B17',
 'MRAcquisitionType': '2D',
 'SeriesDescription': 'fieldmap56_DeltaTE1p02ms',
 'ProtocolName': 'fieldmap56_DeltaTE1p02ms',
 'ScanningSequence': 'GR',
 'SequenceVariant': 'SP',
 'SequenceName': '_fm2d2r',
 'ImageType': ['ORIGINAL', 'PRIMARY', 'P', 'ND'],
 'SeriesNumber': 6,
 'AcquisitionTime': '08:58:10.132500',
 'AcquisitionNumber': 1,
 'SliceThickness': 3.2,
 'SpacingBetweenSlices': 3.2,
 'SAR': 0.0520705,
 'EchoNumber': 2,
 'EchoTime': 0.00702,
 'RepetitionTime': 1.5,
 'FlipAngle': 72,
 'PartialFourier': 1,
 'BaseResolution': 1

In [12]:
bids_root = './data/raw/bids'

T1map_jsons = glob(os.path.join(bids_root, '*', '*', '*', '*_T1map.json'))
for T1map_json in T1map_jsons:
    inv_1 = os.path.basename(T1map_json).replace('_T1map.json', '_inv-1_MPRAGE' + ext)
    inv_2 = os.path.basename(T1map_json).replace('_T1map.json', '_inv-2_MPRAGE' + ext)

    json_dict_t1map =  {
        "BasedOn": [inv_1, inv_2],
        "EstimationMethod":"Marques et al., 2013"
    }
    
    # save this dict for both the T1map and the T1w images (these dicts are the same)
    with open(T1map_json, 'w') as f:
        json.dump(json_dict_t1map, f)
    with open(T1map_json.replace('T1map', 'T1w'), 'w') as f:
        json.dump(json_dict_t1map, f)

    # TO DO: get excitation repetition time, inversion repetition time, flip angles    
    
# Add task description to functional jsons
func_jsons = glob(os.path.join(bids_root, '*', '*', '*', '*_bold.json'))
for func_json in func_jsons:
    with open(func_json, 'r') as f:
        this_json = json.load(f)
    this_json['TaskName'] = 'Stop signal task'
    this_json['CogAtlasID'] = 'tsk_4a57abb949e1a'
    
    with open(func_json, 'w') as f:
        json.dump(this_json, f)
        
fmap_jsons = glob(os.path.join(bids_root, '*/*/*/*_phasediff.json'))
for fmap_json in fmap_jsons:
    with open(fmap_json, 'r') as f:
        this_json = json.load(f)
        
    this_json['EchoTime1'] = 0.006  # from the magnitude1 json
    this_json['EchoTime2'] = 0.00702 # from the magnitude2 json
    regx = re.compile('.*sub-(?P<sub>\d+)_ses-(?P<ses>[a-z]+)_*')
    regx_dict = regx.match(fmap_json).groupdict()
    if regx_dict['ses'] == 'se':
        this_json['IntendedFor'] = ["ses-%s/func/sub-%s_ses-%s_task-stop_run-%d_bold%s" %(regx_dict['ses'], regx_dict['sub'], regx_dict['ses'], x, ext) for x in [1,2,3]]
    else:
#        this_json['IntendedFor'] = ["ses-%s/func/sub-%s_ses-%s_task-stop_run-%d_bold%s" %(regx_dict['ses'], regx_dict['sub'], regx_dict['ses'], x, ext) for x in [1,2,3]]
        this_json['IntendedFor'] = ["ses-%s/func/sub-%s_ses-%s_task-stop_run-%d_echo-%d_bold%s" %(regx_dict['ses'], regx_dict['sub'], regx_dict['ses'], x, y, ext) for x in [1,2,3] for y in [1,2,3]]
    
    with open(fmap_json, 'w') as f:
        json.dump(this_json, f)

In [13]:
with open('./data/raw/bids/sub-01/ses-me/fmap/sub-01_ses-me_phasediff.json', 'r') as f:
    d = json.load(f)
d

{'Modality': 'MR',
 'MagneticFieldStrength': 6.98,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Investigational_Device_7T',
 'InstitutionName': 'MPI_Leipzig',
 'InstitutionalDepartmentName': 'Department',
 'InstitutionAddress': 'Stephanstrasse_1a_Leipzig-FCAF96-_Leipzig_DE_04103',
 'DeviceSerialNumber': '18919',
 'StationName': 'SEPTEMSYS',
 'PatientPosition': 'HFS',
 'ProcedureStepDescription': 'TA_VB17_FATAG1',
 'SoftwareVersions': 'syngo_MR_B17',
 'MRAcquisitionType': '2D',
 'SeriesDescription': 'fieldmap56_DeltaTE1p02ms',
 'ProtocolName': 'fieldmap56_DeltaTE1p02ms',
 'ScanningSequence': 'GR',
 'SequenceVariant': 'SP',
 'SequenceName': '_fm2d2r',
 'ImageType': ['ORIGINAL', 'PRIMARY', 'P', 'ND'],
 'SeriesNumber': 6,
 'AcquisitionTime': '08:58:10.132500',
 'AcquisitionNumber': 1,
 'SliceThickness': 3.2,
 'SpacingBetweenSlices': 3.2,
 'SAR': 0.0520705,
 'EchoNumber': 2,
 'EchoTime': 0.00702,
 'RepetitionTime': 1.5,
 'FlipAngle': 72,
 'PartialFourier': 1,
 'BaseResolution': 1

In [9]:
bids_root = './data/raw/bids'
dataset_descr = {'Name': 'IMCN Stop Task fMRI protocol comparison',
                  'BIDSVersion': "1.51.0.0-rc1",
                  'Authors': ['Steven Miletic',
                              'Pierre-Louis Bazin',
                              'Nikolaus Weiskopf',
                              'Wietske van der Zwaag',
                              'Birte U. Forstmann',
                              'Robert Trampel'],
                  'HowToAckowledge': 'Please cite the corresponding paper.'}
import json
with open(os.path.join(bids_root, 'dataset_description.json'), 'w') as f:
    json.dump(dataset_descr, f)